## Install necessary libraries

In [ ]:
!pip install qiskit qiskit_aer
!pip install qiskit_ibm_runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.

## Import the libraries

In [ ]:
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import hmac
import hashlib


## Functions To generate shared bases
#### Based on the paper

In [ ]:
import hmac
import hashlib

len_message = 16  # Length of binary output

def hmac_prf(secret_key: str, message: bytes) -> str:
    """
    Generate HMAC using SHA-256 and return a binary output truncated to `len_message` bits.

    Args:
    - secret_key (str): Shared secret key.
    - message (bytes): Input message (previous value + seed + counter).

    Returns:
    - str: Binary string representation (16 bits).
    """
    hmac_output = hmac.new(secret_key.encode(), message, hashlib.sha256).hexdigest()
    binary_output = bin(int(hmac_output, 16))[2:].zfill(256)[:len_message]  # Convert to binary and truncate
    return binary_output

def binary_concat(*args) -> bytes:
    """
    Concatenate multiple binary elements.

    Args:
    - *args: List of elements (str or bytes).

    Returns:
    - bytes: Concatenated result.
    """
    return b''.join(arg.encode() if isinstance(arg, str) else arg for arg in args)

def update_key_and_seed(K: str, S: str) -> tuple:
    """
    Generate a new K and S using HMAC-PRF to ensure forward secrecy.

    Args:
    - K (str): Previous secret key.
    - S (str): Previous seed.

    Returns:
    - (new_K, new_S): Tuple of updated K and S as hexadecimal strings.
    """
    new_K = hmac_prf(K, b"update_key")
    new_S = hmac_prf(S, b"update_seed")
    return new_K, new_S

def generate_next_T(K: str, S: str, prev_T: str, round_num: int) -> str:
    """
    Generate the next T value dynamically based on the previous T.

    Args:
    - K (str): Current secret key.
    - S (str): Current seed.
    - prev_T (str): Previous T value.
    - round_num (int): Round number for counter.

    Returns:
    - str: New T value in binary.
    """
    counter = round_num.to_bytes(1, byteorder='big')
    next_T = hmac_prf(K, binary_concat(prev_T.encode(), S.encode(), counter))
    return next_T



## Round 1

In [ ]:
# === Initialization ===
K = "shared_secret_key"
S = "shared_seed"
prev_T = "0000000000000000"

# === Run a single round and show changes ===
round_number = 1
print(f"Round {round_number}:")
print(f"  Initial K  = {K}")
print(f"  Initial S  = {S}")

# Generate new T
new_T = generate_next_T(K, S, prev_T, round_number)
print(f"  Generated T{round_number} = {new_T}")

# Update K and S for the next round
K, S = update_key_and_seed(K, S)
print(f"  Updated K  = {K}")
print(f"  Updated S  = {S}")

Round 1:
  Initial K  = shared_secret_key
  Initial S  = shared_seed
  Generated T1 = 0100100001110000
  Updated K  = 1100100011101110
  Updated S  = 0111010110100001


In [ ]:
round_number = 2
print(f"Round {round_number}:")
print(f"  Initial K  = {K}")
print(f"  Initial S  = {S}")


new_T = generate_next_T(K, S, prev_T, round_number)
print(f"  Generated T{round_number} = {new_T}")

# Update K and S for the next round
K, S = update_key_and_seed(K, S)
print(f"  Updated K  = {K}")
print(f"  Updated S  = {S}")


Round 2:
  Initial K  = 1100100011101110
  Initial S  = 0111010110100001
  Generated T2 = 0000000011000001
  Updated K  = 0000101111100101
  Updated S  = 1001000010010111


## Select common bases

In [ ]:
alice_bases = bob_bases = new_T

## Runtime initialisation

In [ ]:
token= "90c11c3427d32b9170daa73f5fffe26dc68634b8b6f0d8e5a83082726bd42c2d384e16c5ea3f3ed03f5cb71156de8fb4ba9b3880f9f990c6ed2d275f315de5d3"
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

QiskitRuntimeService.save_account(
  token=token,
  channel="ibm_quantum",
  overwrite=True
)


In [ ]:
service = QiskitRuntimeService()
n_qubits = 5
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=n_qubits)
print("Selected Backend", backend)

Selected Backend <IBMBackend('ibm_sherbrooke')>


In [ ]:
print(backend)

<IBMBackend('ibm_sherbrooke')>


## Quantum Random Number Generation

In [ ]:
# Function to generate a random binary string using quantum circuits
def RandomStringIBM(str_len):
    op_str = ''  # Initialize an empty output string
    num_qbits = 16  # Define number of qubits

    # Calculate the number of chunks needed
    num_chunks = math.ceil(str_len / num_qbits)
    for _ in range(num_chunks):
        # Create a quantum register and a classical register
        q = QuantumRegister(num_qbits)
        c = ClassicalRegister(num_qbits)
        QC = QuantumCircuit(q, c)

        # Apply Hadamard gates to all qubits to create superposition
        for i in range(num_qbits):
            QC.h(q[i])
        QC.measure(q, c)
        pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
        isa_circuit = pm.run(QC)

        sampler = Sampler(backend)
        job = sampler.run([isa_circuit])
        print(job.job_id)
        result = job.result()


In [ ]:
len_message = 16
alice_key = RandomStringIBM(len_message)

<bound method BasePrimitiveJob.job_id of <RuntimeJobV2('cyctrvvcw2k0008kns7g', 'sampler')>>


In [ ]:
print(alice_key)

1010000100011111


### Job ID

Random Number : cyctrvvcw2k0008kns7g


In [ ]:
job_id = 'cyctrvvcw2k0008kns7g'
job = service.job(job_id)
result = job.result()
data = result[0].data
bitarray = next(iter(data.values()))
counts = bitarray.get_counts()
alice_key = next(iter(counts.keys()))


In [ ]:
print(alice_key)

1010000100011111


In [ ]:
# Quantum Circuit for encoding
q = QuantumRegister(len_message)
c = ClassicalRegister(len_message)
qc = QuantumCircuit(q, c)

# Encode qubits based on Alice's key and basis
for i in range(len_message):
    if alice_key[i] == '1':
        qc.x(q[i])  # Flip the qubit to 1 if key is 1
    if alice_bases[i] == '1':

        qc.h(q[i])  # Apply Hadamard gate if basis is 1
qc.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=16, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(16, 'q4'), 0), Qubit(QuantumRegister(16, 'q4'), 1), Qubit(QuantumRegister(16, 'q4'), 2), Qubit(QuantumRegister(16, 'q4'), 3), Qubit(QuantumRegister(16, 'q4'), 4), Qubit(QuantumRegister(16, 'q4'), 5), Qubit(QuantumRegister(16, 'q4'), 6), Qubit(QuantumRegister(16, 'q4'), 7), Qubit(QuantumRegister(16, 'q4'), 8), Qubit(QuantumRegister(16, 'q4'), 9), Qubit(QuantumRegister(16, 'q4'), 10), Qubit(QuantumRegister(16, 'q4'), 11), Qubit(QuantumRegister(16, 'q4'), 12), Qubit(QuantumRegister(16, 'q4'), 13), Qubit(QuantumRegister(16, 'q4'), 14), Qubit(QuantumRegister(16, 'q4'), 15)), clbits=())

In [ ]:
# Step 3: Bob measures the qubits
for i in range(len_message):
    if bob_bases[i] == '1':
        qc.h(q[i])  # Bob applies Hadamard if measuring in Hadamard basis

# Add measurements
qc.measure(q, c)

In [ ]:
qc.draw()

┌───┐┌───┐ ░ ┌───┐                     ┌─┐                        
 q4_0: ┤ X ├┤ H ├─░─┤ H ├─────────────────────┤M├────────────────────────
       └───┘└───┘ ░ └───┘┌─┐                  └╥┘                        
 q4_1: ───────────░──────┤M├───────────────────╫─────────────────────────
       ┌───┐┌───┐ ░ ┌───┐└╥┘                   ║ ┌─┐                     
 q4_2: ┤ X ├┤ H ├─░─┤ H ├─╫────────────────────╫─┤M├─────────────────────
       ├───┤└───┘ ░ ├───┤ ║                    ║ └╥┘┌─┐                  
 q4_3: ┤ H ├──────░─┤ H ├─╫────────────────────╫──╫─┤M├──────────────────
       └───┘      ░ └───┘ ║ ┌─┐                ║  ║ └╥┘                  
 q4_4: ───────────░───────╫─┤M├────────────────╫──╫──╫───────────────────
                  ░       ║ └╥┘┌─┐             ║  ║  ║                   
 q4_5: ───────────░───────╫──╫─┤M├─────────────╫──╫──╫───────────────────
       ┌───┐      ░ ┌───┐ ║  ║ └╥┘             ║  ║  ║ ┌─┐               
 q4_6: ┤ H ├──────░─┤ H ├─╫──╫──╫──────────────╫──╫──╫─┤M├───────────────
       ├───┤      ░ └───┘ ║  ║  ║ ┌─┐          ║  ║  ║ └╥┘               
 q4_7: ┤ X ├──────░───────╫──╫──╫─┤M├──────────╫──╫──╫──╫────────────────
       └───┘      ░       ║  ║  ║ └╥┘┌─┐       ║  ║  ║  ║                
 q4_8: ───────────░───────╫──╫──╫──╫─┤M├───────╫──╫──╫──╫────────────────
                  ░       ║  ║  ║  ║ └╥┘┌─┐    ║  ║  ║  ║                
 q4_9: ───────────░───────╫──╫──╫──╫──╫─┤M├────╫──╫──╫──╫────────────────
       ┌───┐      ░ ┌───┐ ║  ║  ║  ║  ║ └╥┘    ║  ║  ║  ║ ┌─┐            
q4_10: ┤ H ├──────░─┤ H ├─╫──╫──╫──╫──╫──╫─────╫──╫──╫──╫─┤M├────────────
       ├───┤┌───┐ ░ ├───┤ ║  ║  ║  ║  ║  ║     ║  ║  ║  ║ └╥┘┌─┐         
q4_11: ┤ X ├┤ H ├─░─┤ H ├─╫──╫──╫──╫──╫──╫─────╫──╫──╫──╫──╫─┤M├─────────
       ├───┤├───┤ ░ ├───┤ ║  ║  ║  ║  ║  ║     ║  ║  ║  ║  ║ └╥┘┌─┐      
q4_12: ┤ X ├┤ H ├─░─┤ H ├─╫──╫──╫──╫──╫──╫─────╫──╫──╫──╫──╫──╫─┤M├──────
       ├───┤├───┤ ░ ├───┤ ║  ║  ║  ║  ║  ║     ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
q4_13: ┤ X ├┤ H ├─░─┤ H ├─╫──╫──╫──╫──╫──╫─────╫──╫──╫──╫──╫──╫──╫─┤M├───
       ├───┤└───┘ ░ └───┘ ║  ║  ║  ║  ║  ║ ┌─┐ ║  ║  ║  ║  ║  ║  ║ └╥┘   
q4_14: ┤ X ├──────░───────╫──╫──╫──╫──╫──╫─┤M├─╫──╫──╫──╫──╫──╫──╫──╫────
       ├───┤┌───┐ ░ ┌───┐ ║  ║  ║  ║  ║  ║ └╥┘ ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐
q4_15: ┤ X ├┤ H ├─░─┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
       └───┘└───┘ ░ └───┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
c1: 16/═══════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                          1  4  5  7  8  9  14 0  2  3  6  10 11 12 13 15

In [ ]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

sampler = Sampler(backend)
job = sampler.run([isa_circuit],shots=1)
result = job.result()


In [ ]:
data = result[0].data
bitarray = next(iter(data.values()))
counts = bitarray.get_counts()

## Run the circuit

Circuit : cyctsyf01rbg008k06yg

In [ ]:

job_id = 'cyctsyf01rbg008k06yg'
job = service.job(job_id)
result = job.result()
data = result[0].data
bitarray = next(iter(data.values()))
counts = bitarray.get_counts()
bob_result = next(iter(counts.keys()))


In [ ]:
bob_result = next(iter(counts.keys()))
print(bob_result)

1111100011000101


In [ ]:
print((alice_bases))
print((bob_bases))
print((alice_key))
print((bob_result))


1011001000111101
1011001000111101
1010000100011111
1111100011000101


In [ ]:
# Display matching bits for shared key
shared_key = []
for i in range(len_message):
    if alice_bases[i] == bob_bases[i]:  # Only keep the bit if bases match
        shared_key.append(bob_result[len_message - i - 1])  # Reverse order in Qiskit

final_shared_key = ''.join(shared_key)
print('Final Shared Key:', final_shared_key)

# Calculate the percentage of key bits retained
matching_bits_count = len(final_shared_key)
percentage_retained = (matching_bits_count / len_message) * 100
print("Percentage of key bits retained: ", percentage_retained, "%")

Final Shared Key: 1010001100011111
Percentage of key bits retained:  100.0 %


## One time pad

In [34]:
final_shared_key = 1010001100011111

fin

In [35]:
import random

# Function to convert a string to binary
def text_to_bin(text):
    return ''.join(format(ord(c), '08b') for c in text)

# Function to convert binary back to string
def bin_to_text(binary_string):
    chars = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]
    return ''.join(chr(int(char, 2)) for char in chars)

# Function to encrypt a message using One-Time Pad
def encrypt(message, key):
    # Convert the message to binary
    binary_message = text_to_bin(message)

    # Encrypt by XORing the binary message with the key (key length fixed at 28)
    encrypted = ''.join(str(int(binary_message[i]) ^ int(key[i % len(key)])) for i in range(len(binary_message)))
    return encrypted

# Function to decrypt a message using One-Time Pad
def decrypt(ciphertext, key):
    # Decrypt by XORing the ciphertext with the key
    decrypted = ''.join(str(int(ciphertext[i]) ^ int(key[i % len(key)])) for i in range(len(ciphertext)))

    # Convert the decrypted binary string back to text
    return bin_to_text(decrypted)

# Example usage
message = "Helod World"

# Assuming final_shared_key is predefined and has a length of 28

print(f"Key: {final_shared_key}")

# Encrypt the message using One-Time Pad
ciphertext = encrypt(message, final_shared_key)
print(f"Encrypted Message (Binary): {ciphertext}")

# Decrypt the message using the same key
decrypted_message = decrypt(ciphertext, final_shared_key)
print(f"Decrypted Message: {decrypted_message}")


Key: 1010001100011111


TypeError: object of type 'int' has no len()